# Creating/Using Image Classifier


### Step 1 - Create Data for Classifier

In [1]:
import cv2

### Load HAAR Classifier


In [2]:
classifier = cv2.CascadeClassifier('cascade/haarcascade_frontalface_default.xml')

### Create a function to extract the face of the user

In [3]:
def face_extractor(image):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray_image, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = image[y:y+h, x:x+w]

    return cropped_face

### Open video camera to take pictures of the user

In [4]:
video = cv2.VideoCapture(0)
count = 0

while(True):
    ret, frame = video.read()
    if face_extractor(frame) is not None:
        count = count + 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = 'faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), 
                    cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
video.release()
cv2.destroyAllWindows()
if (count<30): 
    print("Unable to get user's face. Kindly run this code again.")
else:
    print("Face extraction complete. Now, start training the model.")

Face extraction complete. Now, start training the model.


### Step 2 - Train Model

In [5]:
import numpy as np
from os import listdir
from os.path import isfile, join

In [6]:
# Get the training data we previously made
data_path = 'faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

In [7]:
# Create arrays for training data and labels
training_data, labels = [], []

In [8]:
# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    training_data.append(np.asarray(images, dtype=np.uint8))
    labels.append(i)

In [9]:
# Create a numpy array for both training data and labels
labels = np.asarray(labels, dtype=np.int32)

In [10]:
# Initialize facial recognizer
model = cv2.face.LBPHFaceRecognizer_create()

In [11]:
# Let's train our model 
model.train(np.asarray(training_data), np.asarray(labels))
print("Model trained sucessefully")

Model trained sucessefully


### Step 3 - Save the model to external file

In [12]:
model.save('custom-classifier.xml')